In [3]:
# loading required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
cuisine = raw_input("What do you like to Eat(Cuisine Type):")
budget = raw_input("What's your bugdet for one person:")
location = raw_input("Preferred Location:")
additional = raw_input("Any other comments:")

In [37]:
budget = int(budget)

### Preprocessing and normalization

In [38]:
reviews = pd.read_csv("Data/RestoInfo - Recommendation.csv")

In [39]:
# Restaurant Type
temp_rest_type = reviews[['name','listed_in(type)','listed_in(city)','rest_type']]
temp_rest_type['rest_type'] = temp_rest_type['rest_type'].str.split(',').tolist()
temp_rest_type['rest_type'] = temp_rest_type['rest_type'].fillna('0')

temp_rest_type['key'] = temp_rest_type['name'] + temp_rest_type['listed_in(type)'] + temp_rest_type['listed_in(city)']
temp_rest_type1 = temp_rest_type[['key','rest_type']]

rest_type = pd.DataFrame([[x] + [z] for x, y in temp_rest_type1.values for z in y],columns=temp_rest_type1.columns)

rest_type = rest_type.merge(temp_rest_type[['key','name','listed_in(type)','listed_in(city)']],how = 'left',on = 'key')
rest_type = rest_type[['key','name','listed_in(type)','listed_in(city)','rest_type']]

In [40]:
# Liked Dishes
temp_dish_liked = reviews[['name','listed_in(type)','listed_in(city)','dish_liked']]
temp_dish_liked['dish_liked'] = temp_dish_liked['dish_liked'].str.split(',').tolist()
temp_dish_liked['dish_liked'] = temp_dish_liked['dish_liked'].fillna('0')

temp_dish_liked['key'] = temp_dish_liked['name'] + temp_dish_liked['listed_in(type)'] + temp_dish_liked['listed_in(city)']
temp_dish_liked1 = temp_dish_liked[['key','dish_liked']]

dish_liked = pd.DataFrame([[x] + [z] for x, y in temp_dish_liked1.values for z in y],columns=temp_dish_liked1.columns)

dish_liked = dish_liked.merge(temp_dish_liked[['key','name','listed_in(type)','listed_in(city)']],how = 'left',on = 'key')
dish_liked = dish_liked[['key','name','listed_in(type)','listed_in(city)','dish_liked']]
dish_liked['dish_liked'] = pd.DataFrame([a.strip() for a in dish_liked['dish_liked']])

In [41]:
# cuisines
temp_cuisines = reviews[['name','listed_in(type)','listed_in(city)','cuisines']]
temp_cuisines['cuisines'] = temp_cuisines['cuisines'].str.split(',').tolist()
temp_cuisines['cuisines'] = temp_cuisines['cuisines'].fillna('0')

temp_cuisines['key'] = temp_cuisines['name'] + temp_cuisines['listed_in(type)'] + temp_cuisines['listed_in(city)']
temp_cuisines1 = temp_cuisines[['key','cuisines']]

cuisines = pd.DataFrame([[x] + [z] for x, y in temp_cuisines1.values for z in y],columns=temp_cuisines1.columns)

cuisines = cuisines.merge(temp_cuisines[['key','name','listed_in(type)','listed_in(city)']],how = 'left',on = 'key')
cuisines = cuisines[['key','name','listed_in(type)','listed_in(city)','cuisines']]

In [42]:
# reviews_list
temp_reviews_list = reviews[['name','listed_in(type)','listed_in(city)','reviews_list']]
temp_reviews_list['reviews_list'] = temp_reviews_list['reviews_list'].str.split(',').tolist()
temp_reviews_list['reviews_list'] = temp_reviews_list['reviews_list'].fillna('0')

temp_reviews_list['key'] = temp_reviews_list['name'] + temp_reviews_list['listed_in(type)'] + temp_reviews_list['listed_in(city)']
temp_reviews_list1 = temp_reviews_list[['key','reviews_list']]

reviews_list = pd.DataFrame([[x] + [z] for x, y in temp_reviews_list1.values for z in y],columns=temp_reviews_list1.columns)

reviews_list = reviews_list.merge(temp_reviews_list[['key','name','listed_in(type)','listed_in(city)']],how = 'left',on = 'key')
reviews_list = reviews_list[['key','name','listed_in(type)','listed_in(city)','reviews_list']]

In [43]:
# menu_item
temp_menu_item = reviews[['name','listed_in(type)','listed_in(city)','menu_item']]
temp_menu_item['menu_item'] = temp_menu_item['menu_item'].apply(lambda x: x.replace('[','').replace(']',''))
temp_menu_item['menu_item'] = temp_menu_item['menu_item'].apply(lambda x: x.replace("'",""))
temp_menu_item['menu_item'] = pd.DataFrame([a.strip() for a in temp_menu_item['menu_item']])
temp_menu_item['menu_item'] = temp_menu_item['menu_item'].str.split(',').tolist()
temp_menu_item['menu_item'] = temp_menu_item['menu_item'].fillna('0')

temp_menu_item['key'] = temp_menu_item['name'] + temp_menu_item['listed_in(type)'] + temp_menu_item['listed_in(city)']
temp_menu_item1 = temp_menu_item[['key','menu_item']]

menu_item = pd.DataFrame([[x] + [z] for x, y in temp_menu_item1.values for z in y],columns=temp_menu_item1.columns)

menu_item = menu_item.merge(temp_menu_item[['key','name','listed_in(type)','listed_in(city)']],how = 'left',on = 'key')
menu_item = menu_item[['key','name','listed_in(type)','listed_in(city)','menu_item']]
menu_item['menu_item'] = pd.DataFrame([a.strip() for a in menu_item['menu_item']])

In [44]:
reviews = reviews[['name','online_order','book_table','rate','votes','location','approx_cost(for two people)',
                  'listed_in(type)','listed_in(city)']]

## To calculate score for all restaurant based on user input

### Location

We are going to score based on the following criteria

1. If the store is present in the preferred location - Score 1
2. If the store is not present in the preferred location, but in top 1 location, Score 0.75
3. If the store is not present in the preferred location, but in top 2 location, Score 0.50

These weights are random, can be modified later based on model performance

In [45]:
rest_with_ratings = reviews[~reviews['rate'].isna()]
rest_with_ratings['rate'] = rest_with_ratings['rate'].apply(lambda x: x.replace("/5",""))
rest_with_ratings = rest_with_ratings[rest_with_ratings['rate'] != 'NEW']
rest_with_ratings = rest_with_ratings[rest_with_ratings['rate'] != '-']
rest_with_ratings['rate'] = pd.DataFrame([a.strip() for a in rest_with_ratings['rate']])
rest_with_ratings['rate'] = pd.DataFrame([float(a) for a in rest_with_ratings['rate']])
rest_with_ratings['votes'] = pd.DataFrame([float(a) for a in rest_with_ratings['votes']])

rest_with_ratings['rate x vote'] = rest_with_ratings['rate'] * rest_with_ratings['votes']

location_rating = rest_with_ratings.groupby('listed_in(city)').agg({'rate x vote':'sum','votes':'sum'}).reset_index()
location_rating['weighted_rate'] = location_rating['rate x vote'] / location_rating['votes']
location_rating = location_rating.sort_values(by = 'weighted_rate', ascending = False)

In [46]:
loc_wise_rest_count = rest_with_ratings.groupby('listed_in(city)').agg({'name':'count'}).reset_index()
location_rating = location_rating.merge(loc_wise_rest_count, how = 'left', on= 'listed_in(city)')
location_rating.rename(columns = {'name':'Num of stores'}, inplace = True)
location_rating['Rank'] = location_rating['weighted_rate'].rank(ascending = False)

In [47]:
rank_score_df =pd.DataFrame({'Rank': [1.0,2.0,3.0],
                              'score': [0.75,0.50,0.25]})

In [48]:
location_rating = location_rating.merge(rank_score_df, how = 'left', on = 'Rank')
location_rating.loc[location_rating['score'].isna(), 'score'] = 0 

In [49]:
reviews['location_score'] = reviews['listed_in(city)'].apply(lambda x: 1 if x == location else -99)

In [50]:
reviews = reviews.merge(location_rating[['score','listed_in(city)']], how = 'left', on = 'listed_in(city)')

reviews.loc[(reviews['location_score'] != 1) , 'location_score'] = reviews[reviews['location_score'] != 100]['score']
reviews = reviews.drop(columns = ['score'])

### Cuisine

Doing the same for cuisine

In [51]:
# Cuisine
rest_with_ratings['key'] = rest_with_ratings['name'] + rest_with_ratings['listed_in(type)'] + rest_with_ratings['listed_in(city)']

cuisine_rating_base_data = rest_with_ratings.merge(cuisines[['key','cuisines']],how = 'inner',on = 'key')
cuisine_rating = cuisine_rating_base_data.groupby('cuisines').agg({'rate x vote':'sum','votes':'sum'}).reset_index()
cuisine_rating['weighted_rate'] = cuisine_rating['rate x vote'] / cuisine_rating['votes']
cuisine_rating = cuisine_rating.sort_values(by = 'weighted_rate', ascending = False)

In [52]:
# Number of stores offerig the cuisine
cuisine_wise_store_count = cuisine_rating_base_data.groupby('cuisines').agg({'name':'count'}).reset_index()
cuisine_rating = cuisine_rating.merge(cuisine_wise_store_count, how = 'left', on = ['cuisines'])
cuisine_rating = cuisine_rating.sort_values(by = 'name', ascending = False)

cuisine_rating['Rank'] = cuisine_rating['weighted_rate'].rank(ascending = False)

In [53]:
cuisines = cuisines.merge(cuisine_rating[['cuisines','Rank']], how = 'left', on = 'cuisines')

In [54]:
cuisine_rating = cuisine_rating.merge(rank_score_df, how = 'left', on = 'Rank')
cuisine_rating.loc[cuisine_rating['score'].isna(), 'score'] = 0 

In [55]:
cuisine_rating.loc[(cuisine_rating['cuisines'] == cuisine) , 'score'] = 1

In [56]:
cuisines = cuisines.merge(cuisine_rating[['cuisines','score']],on = 'cuisines', how = 'left')

In [57]:
cuisines.rename(columns = {'score':'cuisine_score'}, inplace = True)

In [58]:
reviews['key'] = reviews['name'] + reviews['listed_in(type)'] + reviews['listed_in(city)']

reviews2 = reviews.merge(cuisines[['key','cuisine_score']], how = 'left', on = 'key')

### Budget

1. If +- 100 then 1
2. If diff >100 & < 200 then 0.75
3. if diff < 500 and > 200 0.5
4. else 0

In [59]:
rev_w_approx_cost_people = reviews[~reviews['approx_cost(for two people)'].isna()][['key','approx_cost(for two people)']]
rev_w_approx_cost_people['approx_cost(for two people)'] = rev_w_approx_cost_people['approx_cost(for two people)'].apply(lambda x: x.replace(",",""))

rev_w_approx_cost_people['approx_cost(for two people)'] = pd.DataFrame([int(i) for i in rev_w_approx_cost_people['approx_cost(for two people)']])

In [60]:
rev_w_approx_cost_people['cost_per_person'] = rev_w_approx_cost_people['approx_cost(for two people)'] /2

In [61]:
rev_w_approx_cost_people['budget_score'] = 0
rev_w_approx_cost_people.loc[(rev_w_approx_cost_people['cost_per_person'] < budget + 500) & (rev_w_approx_cost_people['cost_per_person'] > budget - 500), 'budget_score'] = 0.5
rev_w_approx_cost_people.loc[(rev_w_approx_cost_people['cost_per_person'] < budget + 200) & (rev_w_approx_cost_people['cost_per_person'] > budget - 200), 'budget_score'] = 0.75
rev_w_approx_cost_people.loc[(rev_w_approx_cost_people['cost_per_person'] < budget + 100) & (rev_w_approx_cost_people['cost_per_person'] > budget - 100), 'budget_score'] = 1

In [62]:
reviews2 = reviews2.merge(rev_w_approx_cost_people[['key','budget_score']], how = 'left', on = 'key')

## Final Score



In [63]:
reviews2['final_score'] = ((reviews2['location_score'] * 2) + (reviews2['cuisine_score'] * 2) + (reviews2['budget_score'] * 1)) / 5
# reviews2['final_score'] =                                                                                                        
                                                                                                       

In [64]:
Final_recommednations = reviews2.sort_values(by = 'final_score', ascending = False)[['name','final_score']].head(3).reset_index()


In [65]:
Final_recommednations['name'][2]

'Fujian Express'

In [66]:
print("Restaurants matching your preference:")
print("Top 1: " + Final_recommednations['name'][0] + " (" + str(Final_recommednations['final_score'][0]*100) + "%)")
print("Top 2: " + Final_recommednations['name'][1] + " (" + str(Final_recommednations['final_score'][1]*100) + "%)")
print("Top 3: " + Final_recommednations['name'][2] + " (" + str(Final_recommednations['final_score'][2]*100) + "%)")



Restaurants matching your preference:
Top 1: M.M Restaurant (95.0%)
Top 2: New Delicacy (95.0%)
Top 3: Fujian Express (95.0%)
